# Summarize Text

## Overview

LLMs are a great tool for summarizing text content, given their proficiency in understanding and generating text.

In the context of [RAG (retrieval-augmented generation)](https://github.com/TCLee/rag-langchain), summarizing text can help distill the information in a large number of retrieved documents to provide context for a LLM.

In this notebook, we'll go over how to summarize text content with 3 different strategies. First, we split/chunk a long piece of text into "sub-documents". Then, we'll explore the following strategies:
- **Stuff**: Simply "stuff" all the documents into a single prompt. This is the simplest approach if all the documents can fit inside the model's context window.
- **Map-Reduce**: Summarize each document in a `map` step and then `reduce` the summaries into a final summary. The `map` step is typically _parallelized_ over the input documents.
- **Iterative Refinement**:
    1. Summarize the first document.
    2. Refine/update the result based on the next document.
    3. Repeat through the sequence of documents until finished.

## Run models locally

Before we begin, please make sure you followed the instructions in the [`README.md`](README.md) to setup [`Ollama`](https://ollama.com/) and [`Llama 3.2`](https://ollama.com/library/llama3.2).

Two important benefits of running LLMs locally on your own device are:
- **Privacy**: Your data is not sent to a third party, and it is not subject to the terms of service of a commercial service.
- **Cost**: There is no inference fee, which is important for token-intensive applications (e.g., summarization, agent simulations)

In this notebook, we'll be using `Llama 3.2` with size of `3B` parameters. We'll use `Ollama` to serve the LLM and run inference locally.

For more info, see this LangChain guide: [How to run models locally](https://python.langchain.com/docs/how_to/local_llms/).

Let's load the model into memory and try it out:

In [8]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(
    model="llama3.2:3b", 
    temperature=0
)

chat_model.invoke("Who was the first man on the moon?")

AIMessage(content='The first man to walk on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon\'s surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That\'s one small step for man, one giant leap for mankind," as he became the first person to set foot on another celestial body.\n\nHowever, it\'s worth noting that while Neil Armstrong was the first person to walk on the Moon, Edwin "Buzz" Aldrin also walked on the Moon during the same mission.', additional_kwargs={}, response_metadata={'model': 'llama3.2:3b', 'created_at': '2024-10-04T07:20:08.317072Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 18360137125, 'load_duration': 9171645458, 'prompt_eval_count': 34, 'prompt_eval_duration': 581221000, 'eval_count': 112, 'eval_duration': 8579658000}, id='run-50a90817-4af6-4b9e-823a-fea7fa073872-0', usage_metadata={'input_tokens': 34, 'output_tokens': 112, 'total_

## Document Loading

Let's load in a sample text file that we will use for summarization.

The sample file is a short story by [Edgar Allan Poe](https://en.wikipedia.org/wiki/Edgar_Allan_Poe) titled ["The Cask of Amontillado"](https://www.gutenberg.org/ebooks/1063). This and other short stories are freely available on [Project Gutenberg](https://www.gutenberg.org/).

In [11]:
from langchain_community.document_loaders import TextLoader

text_loader = TextLoader(
    file_path="data/the-cask-of-amontillado.txt", 
    encoding="utf-8"
)
docs = text_loader.load()

In [29]:
print(
    docs[0].page_content[:500]
)

The Cask of Amontillado
by Edgar Allan Poe


The thousand injuries of Fortunato I had borne as I best could, but
when he ventured upon insult, I vowed revenge.  You, who so well know
the nature of my soul, will not suppose, however, that I gave utterance
to a threat.  _At length_ I would be avenged; this was a point definitely
settled--but the very definitiveness with which it was resolved,
precluded the idea of risk.  I must not only punish, but punish with
impunity.  A wrong is unredressed whe


## Stuff: Summarize text in a single LLM call

For models with larger context windows, we can summarize a long document via a single LLM call. Here we use `Llama 3.2` that supports a context length of `128K` tokens.

LangChain implements a simple pre-built chain [`create_stuff_documents_chain`](https://python.langchain.com/api_reference/langchain/chains/langchain.chains.combine_documents.stuff.create_stuff_documents_chain.html)
that "stuffs" a prompt with the desired context for summarization (and other purposes). It takes a list of documents, insert them all into a prompt, and pass that prompt to an LLM.

> See also: [How to summarize text in a single LLM call](https://python.langchain.com/docs/how_to/summarize_stuff/)

In [34]:
from langchain.chains.combine_documents \
    import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# Define prompt
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("human", 
         "Write a concise summary of the following:\n\n"
         "{context}")
    ]
)

# Instantiate chain
chain = create_stuff_documents_chain(
    llm=chat_model, 
    prompt=prompt_template,
    document_variable_name='context',
)

# Invoke chain
result = chain.invoke(
    input={"context": docs}
)

`Llama 3.2` outputs the summary in `Markdown` format. Let's display that in the notebook:

In [36]:
from IPython.display import display_markdown

display_markdown(result, raw=True)

The infamous "Cask of Amontillado" by Edgar Allan Poe, a masterclass in atmospheric horror and psychological manipulation.

The story revolves around Montresor, who seeks revenge against his acquaintance Fortunato for an unspecified offense. Montresor lures Fortunato into the catacombs beneath his family's castle with promises of rare wine, only to trap him alive and ultimately kill him by walled him up in a tomb.

Throughout the narrative, Poe expertly crafts tension and suspense through:

1. **Atmosphere**: The damp, musty air of the catacombs creates an eerie setting that immerses the reader.
2. **Psychological manipulation**: Montresor's words are laced with sarcasm, irony, and condescension, making Fortunato (and the reader) feel uneasy and complicit in his own demise.
3. **Building tension**: Poe skillfully ratchets up the suspense by introducing new sounds, movements, and emotions, keeping the reader on edge.
4. **Symbolism**: The Amontillado wine serves as a symbol of Montresor's revenge, while the catacombs represent the darkness within human nature.

The story's climax is both shocking and chilling, leaving the reader with a sense of unease and a deeper understanding of the complexities of human psychology.

Do you have any specific questions about this tale or would you like to discuss its themes and symbolism?

### Streaming

Note that we can also stream the result token-by-token:

In [26]:
input_dict = {"context": docs}

for token in chain.stream(input=input_dict):
    print(token, end="|")

The| infamous| "|C|ask| of| Am|ont|ill|ado|"| by| Edgar| Allan| Poe|,| a| master|class| in| atmospheric| horror| and| psychological| manipulation|.

|The| story| revolves| around| Mont|res|or|,| who| seeks| revenge| against| his| acquaintance| Fort|un|ato| for| an| unspecified| offense|.| Mont|res|or| l|ures| Fort|un|ato| into| the| cata|com|bs| beneath| his| family|'s| castle| with| promises| of| rare| wine|,| only| to| trap| him| alive| and| ultimately| kill| him| by| w|alled| him| up| in| a| tomb|.

|Throughout| the| narrative|,| Poe| expert|ly| crafts| tension| and| suspense| through|:

|1|.| **|At|mos|phere|**:| The| damp|,| must|y| air| of| the| cata|com|bs| creates| an| eerie| setting| that| imm|ers|es| the| reader|.
|2|.| **|Psych|ological| manipulation|**:| Mont|res|or|'s| words| are| l|aced| with| sarc|asm|,| irony|,| and| con|desc|ension|,| making| Fort|un|ato| (|and| the| reader|)| feel| uneasy| and| comp|licit| in| his| own| demise|.
|3|.| **|Building| tension|**:| Poe| sk